In [1]:
import pandas as pd
import numpy as np
import itertools
from scipy.spatial import distance

pd.set_option('display.max_rows', 500)

### Create inputs

### Populate dataframe

In [2]:
def fill_dataframe(objects, objects_at_once=2):
    df = pd.DataFrame(columns=['from_node', 'to_node', 'obj_at_once', 'dist', 'weight_new'])
    # Generate list of combinations w/o duplicates
    nodes = list(itertools.chain(*[itertools.combinations(objects, i+1) for i in range(len(objects))]))
    nodes = [''.join(node) for node in nodes]
    
    node_list = []
    for x in range(0, len(nodes)):
        if len(nodes[x]) <= 2:
            df.loc[x, 'from_node'] = 'z'
            df.loc[x, 'to_node'] = nodes[x]
            df.loc[x, 'obj_at_once'] = len(nodes[x])
    
        for y in range(x, len(nodes)):
            if 1 <= (len(nodes[y]) - len(nodes[x])) <= 2:
                if all(x in nodes[y] for x in nodes[x]):
                    node_list.append((nodes[y], nodes[x]))
    
    for tup in node_list:
        df = df.append({'from_node': tup[1], 'to_node': tup[0], 'obj_at_once': len(tup[0]) - len(tup[1])}, ignore_index=True)
    
    # No constraints for how many objects at once
    if objects_at_once == 2:
        df = df
    elif objects_at_once == 1:
        # Add constraint: Only 1 object at once
        df = df.loc[df['obj_at_once'] == 1]

    # Reset index
    df = df.reset_index(drop=True)
    
    return df

### Calculate distances

In [3]:
def calculate_distances(data):
    for row in range(0, len(data)):
        # Distance = start -> obj 1 -> table
        if data['from_node'][row] == 'z' and len(data['to_node'][row]) == 1:
            data.loc[row, 'dist'] = (distance.euclidean(
                        coordinates['start'], 
                        coordinates[data['to_node'][row]]) +
                    distance.euclidean(
                        coordinates[data['to_node'][row]], 
                        coordinates['table'])
                    )
    
        # Distance = start -> obj 1 -> obj 2 -> table
        elif data['from_node'][row] == 'z' and len(data['to_node'][row]) != 1:
            data.loc[row, 'dist'] = (distance.euclidean(
                        coordinates['start'], 
                        coordinates[data['to_node'][row][0]]) +
                    distance.euclidean(
                        coordinates[data['to_node'][row][0]], 
                        coordinates[data['to_node'][row][1]]) +
                    distance.euclidean(
                        coordinates[data['to_node'][row][1]], 
                        coordinates['table'])
                    )
    # Distance = table -> obj 1 -> (obj 2) -> table
        else:
            # Get difference between sequences (from_node, to_node)
            diff = [x for x in data['to_node'][row] if x not in data['from_node'][row]]
        
            if len(diff) == 1:
                data.loc[row, 'dist'] = distance.euclidean(
                        coordinates['table'], 
                        coordinates[diff[0]]) * 2
        
            elif len(diff) == 2:
                data.loc[row, 'dist'] = (distance.euclidean(
                        coordinates['table'], 
                        coordinates[diff[0]]) +
                    distance.euclidean(
                        coordinates[diff[0]], 
                        coordinates[diff[1]]) +
                    distance.euclidean(
                        coordinates[diff[1]], 
                        coordinates['table'])
                    )
    return data

In [4]:
def calculate_edge_weights_params(data, objects, c, k):
    # Reset weights according to weight parameters
    for row in range(0, len(data)):
        diff = [x for x in data['to_node'][row] if x not in data['from_node'][row]]
        
        for obj in objects:
            if len(diff) == 1 and obj in diff:
                data.loc[row, 'weight_new'] = (data['dist'][row] ** k[diff[0]]) * c[diff[0]]
                
            elif len(diff) == 2 and obj in diff:
                data.loc[row, 'weight_new'] = (data['dist'][row] ** k[diff[0]]) * c[diff[0]]
                data.loc[row, 'weight_new'] = (data['weight_new'][row] ** k[diff[1]]) * c[diff[1]]
                
    return data

In [58]:
objects = objects = ['p', 'c', 's', 'b', 'a', 'o', 'w', 'e', 'g']

coordinates = {'p': (0,2,3),
               'c': (0,0,3),
               's': (0,0,2),
               'b': (0,0,2),
               'a': (0,0,2),
               'o': (0,0,2),
               'w': (1,0,0),
               'e': (1,0,0),
               'g': (0,2,3),
              'start': (0,3,2),
              'table': (1,1,2)}

c1 = {'p': 1.2,
    'c': 1.2,
    's': 1.2,
    'b': 1.0,
    'a': 1.0,
    'o': 1.0,
    'w': 1.2,
    'e': 1.2,
    'g': 1.2}

k1 = {'p': 0.95,
     'c': 1.0,
     's': 1.0,
     'b': 1.0,
     'a': 1.0,
     'o': 1.0,
     'w': 1.0,
     'e': 1.0,
     'g': 1.0}

c0 = {'p': 1.0,
    'c': 1.0,
    's': 1.0,
    'b': 1.0,
    'a': 1.0,
    'o': 1.0,
    'w': 1.0,
    'e': 1.0,
    'g': 1.0}

k0 = {'p': 1.0,
     'c': 1.0,
     's': 1.0,
     'b': 1.0,
     'a': 1.0,
     'o': 1.0,
     'w': 1.0,
     'e': 1.0,
     'g': 1.0}

In [59]:
def get_first(data, objects):
    possible_items = {}
    
    for row in range(0, len(data)):
        for obj in objects:
            if data.loc[row, 'to_node'] == obj:
                possible_items[obj] = data.loc[row, 'weight_new']
                
    minval = min(possible_items.values())
    list_minval = [k for k, v in possible_items.items() if v==minval]
    
    return possible_items, list_minval

In [90]:
#data2 = fill_dataframe(objects, objects_at_once=1)
#data2 = calculate_distances(data2)

data2 = calculate_edge_weights_params(data2, objects, c0, k1)

get_first(data2, objects)

({'p': 2.97101915279473,
  'c': 4.894328467737257,
  's': 4.414213562373095,
  'b': 4.414213562373095,
  'a': 4.414213562373095,
  'o': 4.414213562373095,
  'w': 5.977725364273731,
  'e': 5.977725364273731,
  'g': 3.1462643699419726},
 ['p'])

In [61]:
# starting points after action x
start0 = (0,1,2)
start1 = (1,1,2)
start2 = (1,1,2)
start3 = (1,1,2)
start4 = (1,1,2)
start5 = (1,1,2)
start6 = (1,1,2)

In [12]:
def get_next(possible_items, coordinates, c, k, start_coord):
    dist_items = {}
    
    for obj in possible_items:
        dist_items[obj] = ((distance.euclidean(
                        start_coord, 
                        coordinates[obj]) +
                    distance.euclidean(
                        coordinates[obj], 
                        coordinates['table'])
                    ) ** k[obj]) * c[obj]
        
    minval = min(dist_items.values())
    list_minval = [k for k, v in dist_items.items() if v==minval]
        
    return dist_items, list_minval
    

In [91]:
items = ['c', 's', 'b', 'a', 'o', 'w', 'e', 'g']
start_coord = start1

get_next(items, coordinates, c0, k1, start_coord)

({'c': 3.4641016151377544,
  's': 2.8284271247461903,
  'b': 2.8284271247461903,
  'a': 2.8284271247461903,
  'o': 2.8284271247461903,
  'w': 4.47213595499958,
  'e': 4.47213595499958,
  'g': 3.4641016151377544},
 ['s', 'b', 'a', 'o'])

In [93]:
items = ['c', 'b', 'a', 'o', 'w', 'e', 'g']
start_coord = start2

get_next(items, coordinates, c0, k1, start_coord)

({'c': 3.4641016151377544,
  'b': 2.8284271247461903,
  'a': 2.8284271247461903,
  'o': 2.8284271247461903,
  'w': 4.47213595499958,
  'e': 4.47213595499958,
  'g': 3.4641016151377544},
 ['b', 'a', 'o'])

In [94]:
items = ['c', 'a', 'o', 'w', 'e', 'g']
start_coord = start3

get_next(items, coordinates, c0, k1, start_coord)

({'c': 3.4641016151377544,
  'a': 2.8284271247461903,
  'o': 2.8284271247461903,
  'w': 4.47213595499958,
  'e': 4.47213595499958,
  'g': 3.4641016151377544},
 ['a', 'o'])

In [95]:
items = ['c', 'o', 'w', 'e', 'g']
start_coord = start4

get_next(items, coordinates, c0, k1, start_coord)

({'c': 3.4641016151377544,
  'o': 2.8284271247461903,
  'w': 4.47213595499958,
  'e': 4.47213595499958,
  'g': 3.4641016151377544},
 ['o'])

In [96]:
items = ['c', 'g', 'e', 'w']
start_coord = start5

get_next(items, coordinates, c0, k1, start_coord)

({'c': 3.4641016151377544,
  'g': 3.4641016151377544,
  'e': 4.47213595499958,
  'w': 4.47213595499958},
 ['c', 'g'])

In [97]:
items = ['e', 'w', 'g']
start_coord = start6

get_next(items, coordinates, c0, k1, start_coord)

({'e': 4.47213595499958, 'w': 4.47213595499958, 'g': 3.4641016151377544},
 ['g'])

In [98]:
items = ['e', 'w']
start_coord = start6

get_next(items, coordinates, c0, k1, start_coord)

({'e': 4.47213595499958, 'w': 4.47213595499958}, ['e', 'w'])